# Compte-rendu - projet de génération d'un modèle de détection de mouvement de balancier

## TOC
* [Partie 1](#1)
* [Partie 2](#2)
* [Partie 3](#3)
* [Partie 4](#4)
* [Partie 5](#5)
* [Partie 6](#6)
* [Partie 7](#7)
* [Partie 8](#8)
* [Partie 9](#9)
* [Partie 10](#10)

## Initialisation <a class="anchor" id="1"></a>
Import des packages utile

* `pandas` : librairie pour manipuler les données (`DataFrame`)
* `numpy` : librairie mathématique
* `matplotlib` : librairie graphique

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Variables liées à l'environnement

# seed utilisée pour garantir la reproductibilité des résultats
SEED = 42

# Fréquence d'échantillonnage de la carte
SENSORS_SAMPLING_RATE = 50 # Hz

# Chemin d'accès aux données
DATA_PATH = 'data/'

In [ ]:
# fonctions utilitaires

def flatten(list):
    return [item for sublist in list for item in sublist]

## Récupération des données <a class="anchor" id="2"></a>

Nous avons au préalable réalisé des enregistrements de données des capteurs (disponibles dans le dossier `/data`).
Nous avons stocké dans un [Google sheet](https://docs.google.com/spreadsheets/d/1By59dQ56zL_kP0tW9Iyf4FppyEvJtcwnuG4gx1iokpM/edit?usp=sharing) si les captures correspondent à un état de balancier, ou non.

Nous devons traiter cette donnée brute pour la rendre plus compréhensible et interprétable par Keras.

A des fins d'études nous sauvegarderons l'état du dataset dans plusieurs variables:
* `raw_dataset` : données brutes
* `trimed_dataset` : données après avoir éliminé les limites
* `sampled_dataset` : données après avoir été échantillonnées
* `bounded_dateset` : données après avoir été réduites à un certain intervalle

In [ ]:
from os import listdir
from os.path import isfile, join

dataset = []

csvs = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f))]
for filename in csvs:
    dataset.append(pd.read_csv(DATA_PATH + filename))

raw_dataset = dataset.copy()
len(raw_dataset)

### Trim

La première étape est de trim les enregistrements pour ne garder que la partie qui nous intéresse.
Cela est surtout important dans le cas où notre capture représente un mouvement de balancier, car la donnée brute inclut, au début et à la fin, des instants où l'objet ne se balance pas. Cela correspond au temps qu'il s'écoule entre l'activation de la capture et le début du mouvement.

In [ ]:
# Limites des données utiles de chaque fichier .csv
LIMITES = [ [60, 650],   [50, 350],  [100, 600],
            [0, 550],    [0, 550],   [150, 250],
            [0, 700],    [75, 820],  [0, 500],
            [50, 550],   [50, 750],  [50, 900],
            [0, 900],    [50, 1000], [100, 1200],
            [50, 1300],  [50, 900],  [200, 700],
            [0, 60],     [0, 60],    [0, 25],
            [100, 800],  [100, 900], [100, 600],
            [100, 850],  [50, 900],  [100, 1500],
            [100, 2200], [30, 300],  [0, 650],
            [0, 550],    [100, 500], [0, 710]]

In [ ]:
def trimDataset(ds):
    for i in range(len(ds)):
        ds[i] = ds[i][LIMITES[i][0]:LIMITES[i][1]]

In [ ]:
trimDataset(dataset)

trimed_dataset = dataset.copy()

Le dataset contient maintenant une liste de dataframe de donnes brutes réduites à la partie qui nous intéresse.

### Echantillonnage

Dans cette partie nous allons échantillonner les données, c'est à dire que nous allons réduire le nombre de point de données pour un intervalle de temps donné.

**Raisonnement**

En se basant sur le [théorème de Shannon](), on peut déduire que la fréquence d'échantillonnage nécessaire est deux fois plus grande que la fréquence du signal à détecter.
Dans notre cas on a donc simplement besoin d'un échantillonnage de deux fois la fréquence maximale de balancier.
De par nos expériences on considère que le balancier maximal possible est de 5 Hz.
Notre échantillonnage sera donc de **10 Hz**.

**Intérêt**

cette méthode a deux intérêts :
* Réduire le nombre de points de données, et donc réduire le nombre de neurones d'entrée de notre modèle. Cela permet de réduire le temps de calcul.
* Chaque enregistrement contient plus de données que nécessaire, on peut donc les sous-diviser en plusieurs échantillons. Cela permet d'augmenter la data à notre disposition. Ici on multiplie la taille de notre dataset par 5.

In [ ]:
DATA_SAMPLING_RATE = 10 # Hz

La fonction suivante `sampleDf` permet de sous-diviser une dataframe en plusieurs sous-échantillons.

In [ ]:
def sampleDf(df, scaleFactor):
    res = []
    for i in range(scaleFactor):
        res.append(df.iloc[lambda x: x.index % scaleFactor == i])
    return res

In [ ]:
scalingFactor = SENSORS_SAMPLING_RATE // DATA_SAMPLING_RATE

for i in range(len(dataset)):
    dataset[i] = sampleDf(dataset[i], scalingFactor)

sampled_dataset = flatten(dataset.copy())
len(sampled_dataset)

### Réduction de l'intervalle

Maintenant que nous avons échantillonné les données, nous allons pouvoir maintenant les sous-diviser en intervalles de temps constants.
Ici nous avons choisi de prendre en compte des fenêtre de 2 secondes, soit 20 points de données.

In [ ]:
WINDOW_TIME = 2 # s

WINDOW_LENGTH = WINDOW_TIME * DATA_SAMPLING_RATE

In [ ]:
def sliceDf(df, step):
    res = []
    while (len(df) > step):
        res.append(df.iloc[:step])
        df = df.iloc[1:]
    return res

In [ ]:
for i in range(len(dataset)):
    for ii in range(len(dataset[i])):
        dataset[i][ii] = sliceDf(dataset[i][ii], WINDOW_LENGTH)
    dataset[i] = flatten(dataset[i])

sliced_dataset = flatten(dataset.copy())
len(sliced_dataset)

On souhaite valider les données obtenues en vérifiant que toutes les dataframes obtenues sont bien toutes sur un intervalle de 2 secondes, ie. qu'elles contiennent bien 20 points de données :

In [ ]:
# Vérification de la données
# Toutes les dataframes font-elles bien la même taille ?

sizes = []
for df in sliced_dataset:
    if len(df) not in sizes:
        sizes.append(len(df))
sizes

### Clean des données inutiles

Maintenant on va maintenant faire le choix des colonnes de données inutiles.
Pour rappel voilà les colonnes de données disponibles :

In [ ]:
dataset[0][0].columns

Nous avons choisi de conserver toutes les données disponibles dans un premier temps.
En effet, la colonne du temps n'as pas de sens car un mouvement de balancier doit pouvoir être détecté indépendamment du moment où il apparaît. 

In [ ]:
UNUSED_DATA_COLUMN = ["T [ms]"]

In [ ]:
def cleanDf(df):
    return df.drop(columns=UNUSED_DATA_COLUMN)

In [ ]:
print(dataset[0][0].columns)

for i in range(len(dataset)):
    for ii in range(len(dataset[i])):
        dataset[i][ii] = cleanDf(dataset[i][ii])

print(dataset[0][0].columns)

A l'issue de cette étape, la variable `dataset` contient une liste de dataframe de données traitées et normalisées. Chacune contient une fenêtre de 2 secondes de captures échantillonnés à 10Hz, correspondant à 20 points * 6 capteurs = 120 points de donnée.

Ce sont ces valeurs qui seront passés en entrée de notre modèle pour l'entraîner.

## Labellisation

Maintenant que nous avons normalisé notre dataset, nous allons pouvoir maintenant labelliser les données.
Ce processus consiste à associer à chaque dataframe une valeur numérique représentant le résultat souhaité.

### Fetch des résultats de balancier

On commence par aller récupérer les résultats attendus des différentes captures via le Google Sheet.
Pour cela on a crée un utilitaire `SheetAPI` qui permet de récupérer sous la forme d'un tableau les éléments notés dedans.

In [ ]:
from gSheet import SheetAPI

In [ ]:
# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1By59dQ56zL_kP0tW9Iyf4FppyEvJtcwnuG4gx1iokpM'
api = SheetAPI(SPREADSHEET_ID)
api.connect()

Y = api.getValues("A2:D100")
Ycolumns = Y[0]
Y = Y[0:]

In [ ]:
for i in range(len(Y)):
    Y[i] = [Y[i] for _ in dataset[i]]

In [ ]:
len(Y)

On valide que les sorties contiennent bien le bon nombre de données :

In [ ]:
size_dataset = 0
size_Y = 0

for i in range(len(dataset)):
    size_dataset += len(dataset[i])
    size_Y += len(Y[i])
    assert(len(Y[i]) == len(dataset[i]))

print("size_dataset: ", size_dataset)
print("size_Y: ", size_Y)

### Normalisation de la forme des données

Maintenant que les sorties sont correctement créées, on applatit les deux listes, pour que chaque élément corresponde à une fenêtre de 2 secondes de capture, échantillonné à 10Hz.

In [ ]:
Y = flatten(Y)
dataset = flatten(dataset)

### Création des données de tests

Une fois nos données correctement associées à leurs résultats attendus, nous pouvons maintenant créer des données de tests.
Pour être plus précis, nous allons sortir un tiers de notre dataset pour ne pas l'utiliser comme données d'entraînement, et l'utiliser pour la validation de notre modèle.

In [ ]:
E_train, E_test, Y_train, Y_test = train_test_split(dataset, Y, test_size=0.33, random_state=SEED)

## Génération du modèle

Maintenant que nous avons traité nos données brutes, il faut configurer un modèle capable de les interpréter.


In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model  # install graphviz on OS
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import Adam

### Configuration des paramètres du modèle

In [ ]:
model = Sequential() # Instanciation du modèle

# TODO je suis pas sûr que cette couche soit celle d'entrée, peut-être pas nécessire ?
model.add(Dense(120, input_dim=1, activation='sigmoid')) # Ajout de la couche d'entrée
model.add(Dense(2, activation='sigmoid')) # Ajout de la couche de sortie

# TODO Configuration de l'optimizer
opt = Adam()

model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])

### Training du modèle

In [ ]:
import timeit

start_time = timeit.default_timer()

history = model.fit(E_train, Y_train, validation_split=0.15, shuffle=False, epochs=400, verbose=0, batch_size=5)

print("Temps passé : %.2fs" % (timeit.default_timer() - start_time))

___

___

# Analyse des données
On commence par importer les données depuis un fichier CSV

In [ ]:
d = pd.read_csv('data/old/SensorTile_Log_N008.csv')
d.columns

In [ ]:
def cleanDataframe(df):
    df = df.drop(columns=["T [ms]"])
    return df

def sliceDf(df, step):
    res = []
    while (len(df) > step):
        res.append(df.iloc[:step])
        df = df.iloc[step:]
    return res

In [ ]:
#d = cleanDataframe(d)
d.tail(10)

In [ ]:
dfs = sliceDf(d, 100)
len(d)

In [ ]:
len(dfs)

In [ ]:
plt.plot(d["AccX [mg]"])
plt.plot(d["AccY [mg]"])
plt.plot(d["AccZ [mg]"])

In [ ]:
plt.plot(d["GyroX [mdps]"])

# Parsing the data

Given the data, we can parse it to extract the information we need.

First we slice the dataframe into multiple 1-sec **rolling** windows
Then we multiply the data by sampling the data into subsets.

In [ ]:
SAMPLE_RATE = 50 # Hz
DATA_RATE = 20 # Hz
WINDOW_LENGTH = 2000 # (in ms)

In [ ]:

def sliceDf(df, step):
    res = []
    while (len(df) > step):
        res.append(df.iloc[:step])
        df = df.iloc[1:]
    return res

def removeTime(df):
    df = df.drop(columns=["T [ms]"])
    return df

def sampleDf(df, sample):
    res = []
    for i in range(0, sample): # A tester
        res.append(df.iloc[lambda x: x.index % sample == i])
    return res

In [ ]:
d = pd.read_csv('data/balancier0.csv')
d.columns

In [ ]:
d.tail(10)